In [1]:
# # import re
# from collections import defaultdict, Counter
# # from itertools import groupby
# # from nltk import flatten
# # from nltk.util import trigrams as get_trigrams
# from mwu_measures.preprocessing_corpus import clean_bnc_line, preprocess_bnc
# # import mwu_measures
# from mwu_measures.processing_corpus import Corpus
import mwu_measures
import mwu_measures.processing_corpus_sqlite
from mwu_measures.compute_functions import min_max_norm
from mwu_measures import compute_functions
from mwu_measures import processing_corpus
# from mwu_measures.mwu_functions import get_association, get_entropy_dif
from collections import defaultdict, Counter
# from nltk import FreqDist
import numpy as np
import pandas as pd

In [2]:
this_corpus = mwu_measures.processing_corpus_sqlite.process_corpus(corpus_name = 'bnc', corpus_dir='small_corpus.txt', chunk_size=1000000, verbose=True)

8928 lines processed
16500 lines processed
25755 lines processed
34338 lines processed
42803 lines processed
52217 lines processed
61689 lines processed
71326 lines processed
80991 lines processed
90366 lines processed
98040 lines processed
105999 lines processed
114462 lines processed
123188 lines processed
130358 lines processed
137924 lines processed
147611 lines processed
157164 lines processed
168734 lines processed
178303 lines processed
188473 lines processed
196901 lines processed
206573 lines processed
220860 lines processed
237194 lines processed
250060 lines processed
261943 lines processed
270684 lines processed
278913 lines processed
287810 lines processed
296127 lines processed
301046 lines processed


In [3]:
this_corpus.create_totals()

In [4]:
this_corpus.set_getting_functions()

In [ ]:
this_corpus.get_unigram('most')

AttributeError: 'Corpus' object has no attribute 'get_unigram_distribution'

In [ ]:
get_scratch = this_corpus.corpus_conn.prepare("""
                                 PREPARE get_fw_bigram AS
                                 SELECT corpus, ug_1, ug_2, SUM(freq) AS freq 
                                 FROM trigram_db WHERE ug_1 = $1 
                                 GROUP BY corpus, ug_1, ug_2
                                 """)

In [ ]:
this_corpus.get_fw_distribution("most of")

In [ ]:
mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
mwu_examples = mwu_examples.sample(10000)

In [ ]:
ngram_list = mwu_examples.Expression
bigrams = [ngram.split() for ngram in ngram_list if len(ngram.split()) == 2]
trigrams = [ngram.split() for ngram in ngram_list if len(ngram.split()) == 3]

In [ ]:
trigram_query = pd.DataFrame(trigrams, columns=['ug_1', 'ug_2', 'ug_3'])
this_corpus.corpus_conn.execute("CREATE TEMPORARY TABLE trigram_query (ug_1 TEXT, ug_2 TEXT, ug_3 TEXT)")
this_corpus.corpus_conn.register('trigram_query_list', trigram_query)
this_corpus.corpus_conn.execute("INSERT INTO trigram_query SELECT * FROM trigram_query_list")

In [ ]:
trigram_per_corpus = this_corpus.corpus_conn.execute("""
    SELECT * FROM trigram_db 
    WHERE (ug_1 IN (SELECT ug_1 FROM trigram_query) AND ug_2 IN (SELECT ug_2 FROM trigram_query)) OR (ug_3 IN (SELECT ug_3 FROM trigram_query))
""").fetch_df()
# trigram_total = this_corpus.corpus_conn.execute("""
#     SELECT ug_1, ug_2, ug_3, sum(freq) as freq FROM trigram_db 
#     WHERE (ug_1 IN (SELECT ug_1 FROM trigram_query) AND ug_2 IN (SELECT ug_2 FROM trigram_query)) OR (ug_3 IN (SELECT ug_3 FROM trigram_query))
#     GROUP BY (ug_1, ug_2, ug_3)
# """).fetch_df()

In [ ]:
bigram_query = pd.DataFrame(bigrams, columns=['ug_1', 'ug_2'])
this_corpus.corpus_conn.execute("CREATE TEMPORARY TABLE bigram_query (ug_1 TEXT, ug_2 TEXT)")
this_corpus.corpus_conn.register('bigram_query_list', bigram_query)
this_corpus.corpus_conn.execute("INSERT INTO bigram_query SELECT * FROM bigram_query_list")

In [ ]:
# this_corpus.corpus_conn.execute("DROP TABLE bigram_query")

In [ ]:
bigram_per_corpus = this_corpus.corpus_conn.execute("""
    SELECT corpus, ug_1, ug_2, sum(freq) AS freq FROM trigram_db 
    WHERE (ug_1 IN (SELECT ug_1 FROM bigram_query)) OR (ug_2 IN (SELECT ug_2 FROM bigram_query))
    GROUP BY corpus, ug_1, ug_2
""").fetch_df()
# bigram_total = this_corpus.corpus_conn.execute("""
#     SELECT ug_1, ug_2, sum(freq) AS freq FROM trigram_db 
#     WHERE (ug_1 IN (SELECT ug_1 FROM bigram_query)) OR (ug_2 IN (SELECT ug_2 FROM bigram_query))
#     GROUP BY ug_1, ug_2
# """).fetch_df()

In [ ]:
# takes too long, filter DF first

fw_dists = {ngram_1: {corpus:  Counter(dict(zip(data['ug_2'], data['freq']))) for corpus, data in group.groupby('corpus')} for ngram_1, group in bigram_per_corpus.groupby('ug_1')}

In [ ]:
fw_dists['A']

In [ ]:
this_corpus.corpus_conn.execute("SELECT * FROM trigram_db ORDER BY corpus").fetch_df()

In [ ]:
this_corpus = mwu_measures.processing_corpus.process_corpus(corpus_name='bnc', corpus_dir='small_corpus.txt', verbose=True, chunk_size=1000000)
this_corpus.create_totals()